In [2]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta 
import pandas as pd 
import seaborn as sns

In [3]:
start = datetime(2020,1,18)
date = []
for x in range(0,28):
    day = start + timedelta(30) * x
    day_for = day.strftime('%m/%d/%Y')
    date.append(day_for)


In [4]:
day_list = []
index_list = []
for day in  date:
    html = requests.get(f'http://en.stockbiz.vn/HistoricalIndices.aspx?Symbol=HOSTC&Date={day}').text
    soup = BeautifulSoup(html)
    form = soup.find('form',{'name':'aspnetForm'})
    table = form.find('table')
    div1 = table.find('div',{'id':'mainwrap'})
    div2 = div1.find('div',{'id':'container'})
    div3 = div2.find('div',{'id':'tabcontainer'})
    div4 = div3.find('div',{'id':'centercol'})
    table1 = div4.find('table',{'id':'ctl00_PlaceHolderContentArea_RightZone'})
    tr1 = table1.find_all('tr',{'class':'rowcolor2'})
    tr2 = table1.find_all('tr',{'class':'rowcolor1'})
    for row in tr1:
        day = row.find('td', attrs={'align':'left'}).text.strip()
        day_list.append(day)
        index = row.find_all('td',attrs={'align':'right'})[0].text.strip()
        index_list.append(index)
    for row in tr2:
        day = row.find('td', attrs={'align':'left'}).text.strip()
        day_list.append(day)
        index = row.find_all('td',attrs={'align':'right'})[0].text.strip()
        index_list.append(index)

In [5]:
df = pd.DataFrame(index_list,columns=['index'])
df['day'] = pd.DataFrame(day_list)
df['day'] = pd.to_datetime(df['day'])


In [6]:
VN30 = ['ACB','BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','KDH','MBB','MSN','MWG','NVL','PDR','PLX','PNJ','POW','SAB','SSI','STB','TCB','TPB','VCB','VHM','VIC','VJC','VNM','VPB','VRE']
for stock in VN30:
    locals()[stock] = []
    for day in  date:
        html = requests.get(f'http://en.stockbiz.vn/Stocks/{stock}/LookupQuote.aspx?Date={day}').text
        soup = BeautifulSoup(html)
        table = soup.find('table')
        div1 = table.find('div',{'id':'mainwrap'})
        div2 = div1.find('div',{'id':'container'})
        table1 = div1.find('table',{'id':'ctl00_PlaceHolderContentArea_CenterZone'})
        tr1 = table1.find_all('tr',{'class':'rowcolor2'})
        tr2 = table1.find_all('tr',{'class':'rowcolor1'})
        for row in tr1:
            stock_price = row.find_all('td',attrs={'align':'right'})[4].text
            locals()[stock].append(stock_price.strip())
        for row in tr2:
            stock_price = row.find_all('td',attrs={'align':'right'})[4].text
            locals()[stock].append(stock_price.strip())

In [7]:
for x in VN30:
    df[x] = pd.DataFrame(locals()[x],columns=[x])
df.head()

,index,day,ACB,BID,BVH,CTG,FPT,GAS,GVR,HDB,...,STB,TCB,TPB,VCB,VHM,VIC,VJC,VNM,VPB,VRE
0,978.96,2020-01-17,23.80,53.50,65.90,24.95,57.60,93.20,11.40,28.55,...,10.60,23.40,21.55,94.50,86.00,114.90,148.20,118.60,21.90,32.60
1,967.56,2020-01-15,23.30,51.00,65.90,23.60,57.00,94.00,11.10,27.75,...,10.25,22.90,21.75,89.30,86.30,115.00,146.00,117.80,21.65,33.20
2,965.84,2020-01-13,22.90,50.10,66.70,23.20,57.10,95.30,11.30,27.90,...,10.40,22.95,21.55,89.40,86.30,114.90,146.50,117.40,20.60,33.75
3,960.15,2020-01-09,22.40,49.65,65.10,22.45,57.60,95.20,11.00,27.50,...,10.15,23.05,21.10,88.70,83.30,114.70,146.20,118.60,20.75,34.00
4,958.88,2020-01-07,22.50,46.70,67.50,21.70,58.10,97.10,11.20,26.70,...,10.15,23.20,21.20,87.80,83.80,114.70,145.90,118.50,20.35,33.95


In [8]:
df.sort_values(by='day', inplace=True)
df.reset_index(drop=True)

,index,day,ACB,BID,BVH,CTG,FPT,GAS,GVR,HDB,...,STB,TCB,TPB,VCB,VHM,VIC,VJC,VNM,VPB,VRE
0,963.56,2019-12-06,23.00,39.65,65.80,20.30,55.00,98.00,12.60,27.80,...,10.00,23.15,21.70,84.90,92.00,115.90,144.40,116.30,19.60,34.60
1,966.06,2019-12-09,22.90,40.00,65.70,20.15,55.30,99.80,12.40,28.00,...,9.95,22.80,21.50,85.20,92.00,115.90,144.50,118.10,19.40,35.20
2,960.30,2019-12-10,22.80,39.90,67.70,20.05,55.50,97.90,12.50,27.70,...,10.05,22.40,21.20,85.40,91.50,115.90,143.80,118.20,19.05,34.90
3,961.78,2019-12-11,22.90,40.10,68.00,19.85,56.50,98.10,12.30,28.00,...,10.05,22.70,21.00,85.90,91.10,115.90,143.90,118.60,19.25,35.00
4,968.17,2019-12-12,23.00,41.00,68.00,20.15,57.00,97.90,12.40,28.00,...,10.10,23.15,21.00,87.50,91.20,116.00,145.20,119.20,19.65,34.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,1516.44,2022-04-01,33.40,44.35,63.40,33.05,111.00,109.70,34.55,29.45,...,32.20,50.30,41.25,82.80,76.20,82.10,140.90,81.70,38.60,34.05
836,1524.70,2022-04-04,33.50,44.25,63.60,32.85,109.30,112.40,36.50,29.10,...,32.20,49.65,41.25,82.60,76.30,82.70,142.90,82.20,39.00,34.05
837,1520.03,2022-04-05,33.00,43.30,63.80,32.35,109.00,114.30,36.40,28.75,...,31.80,49.05,40.80,83.00,76.40,82.30,141.50,80.40,38.30,33.65
838,1522.90,2022-04-06,33.20,43.50,65.20,32.60,113.00,113.70,37.00,29.00,...,32.35,49.90,41.15,84.60,76.30,80.50,141.00,79.30,39.95,33.60


In [9]:
df[['index','ACB', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
       'HPG', 'KDH', 'MBB', 'MSN', 'MWG', 'NVL', 'PDR', 'PLX', 'PNJ', 'POW',
       'SAB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE']] = df[['index','ACB', 'BID', 'BVH', 'CTG', 'FPT', 'GAS', 'GVR', 'HDB',
       'HPG', 'KDH', 'MBB', 'MSN', 'MWG', 'NVL', 'PDR', 'PLX', 'PNJ', 'POW',
       'SAB', 'SSI', 'STB', 'TCB', 'TPB', 'VCB', 'VHM', 'VIC', 'VJC', 'VNM',
       'VPB', 'VRE']].astype(float)